In [ ]:
# Neural
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load dataset
df = pd.read_csv('your_dataset.csv')  # Replace with your file
X = df.drop('target', axis=1).values
y = df['target'].values

# Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build model
model = Sequential([
    Dense(64, input_dim=X.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Change to softmax for multiclass
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Simple CNN Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image generator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'path_to_images',  # Replace with your directory
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'path_to_images',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Use softmax for multi-class
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator, validation_data=val_generator, epochs=10)


In [ ]:
# Custom CNN Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'path_to_images',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    'path_to_images',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use softmax for multiclass
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator, validation_data=val_generator, epochs=10)

In [ ]:

# Transfer Learning with VGG16, VGG19, ResNet50, InceptionV3

from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

def build_model(base_model_class, input_shape=(224, 224, 3), num_classes=2):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax' if num_classes > 2 else 'sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy' if num_classes > 2 else 'binary_crossentropy', metrics=['accuracy'])
    return model

# Example usage
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'path_to_images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # or 'binary'
)

val_generator = train_datagen.flow_from_directory(
    'path_to_images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

model = build_model(VGG16, num_classes=train_generator.num_classes)
model.fit(train_generator, validation_data=val_generator, epochs=5)


In [ ]:
#LSTM GRU RNN
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Dense
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('stock_prices.csv')  # Replace with your CSV
data = df['Close'].values.reshape(-1, 1)

# Normalize
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Sequence generation
def create_sequences(data, seq_len=60):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i - seq_len:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled)
X_train, y_train = X[:int(0.8*len(X))], y[:int(0.8*len(X))]
X_test, y_test = X[int(0.8*len(X)):], y[int(0.8*len(X)):]

# Choose model
model_type = 'LSTM'  # Change to 'RNN' or 'GRU'
model = Sequential()
if model_type == 'LSTM':
    model.add(LSTM(50, input_shape=(X.shape[1], 1)))
elif model_type == 'RNN':
    model.add(SimpleRNN(50, input_shape=(X.shape[1], 1)))
elif model_type == 'GRU':
    model.add(GRU(50, input_shape=(X.shape[1], 1)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict
predicted = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted)
actual_prices = scaler.inverse_transform(y_test)

plt.plot(actual_prices, label='Actual')
plt.plot(predicted_prices, label='Predicted')
plt.legend()
plt.show()

In [ ]:
# Next word pred
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Sample text
text = "Deep learning models like LSTM are useful for text prediction tasks such as next word prediction."

# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
sequence = tokenizer.texts_to_sequences([text])[0]

# Create sequences
sequences = []
for i in range(1, len(sequence)):
    seq = sequence[:i+1]
    sequences.append(seq)

# Prepare data
max_seq_len = max(len(seq) for seq in sequences)
X = np.array([np.pad(seq[:-1], (max_seq_len - len(seq[:-1]), 0)) for seq in sequences])
y = to_categorical([seq[-1] for seq in sequences], num_classes=len(tokenizer.word_index)+1)

# Build model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=10, input_length=max_seq_len-1))
model.add(LSTM(50))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=0)

# Predict next word
input_text = "deep learning models"
encoded = tokenizer.texts_to_sequences([input_text])[0]
padded = np.pad(encoded, (max_seq_len - 1 - len(encoded), 0))
predicted_index = np.argmax(model.predict(np.array([padded])), axis=-1)[0]

for word, index in tokenizer.word_index.items():
    if index == predicted_index:
        print(f"Next word prediction: {word}")
        break


In [ ]:
#Vanilla ae
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.
x_train = x_train.reshape((len(x_train), 784))
x_test = x_test.reshape((len(x_test), 784))

input_img = Input(shape=(784,))
encoded = Dense(64, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=20, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

decoded_imgs = autoencoder.predict(x_test)
plt.imshow(decoded_imgs[0].reshape(28, 28), cmap='gray')
plt.title("Reconstructed Image")
plt.show()

In [ ]:
#Deep ae
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.
x_train = x_train.reshape((len(x_train), 784))
x_test = x_test.reshape((len(x_test), 784))

input_img = Input(shape=(784,))
encoded = Dense(512, activation='relu')(input_img)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)

decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=20, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

In [ ]:
#Convolution AE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=10, batch_size=128, shuffle=True, validation_data=(x_test, x_test))

In [ ]:
#Q learning
import numpy as np
import random

# Reward matrix R
R = np.array([
    [-1,  0, -1,  0, -1, -1, -1],
    [ 0, -1,100, -1,  0, -1, -1],
    [-1, -1,-1, 100, -1,  0, -1],
    [ 0, -1,  0, -1, -1, -1,  0],
    [-1,  0, -1, -1, -1,  0, -1],
    [-1, -1,  0, -1,  0, -1,  0],
    [-1, -1,100, -1,  0, -1, -1]
])

# Initialize Q-matrix
Q = np.zeros_like(R)

# Parameters
gamma = 0.8  # discount factor
episodes = 1000

for _ in range(episodes):
    state = random.randint(0, 6)
    while True:
        # Get valid actions from current state
        actions = np.where(R[state] != -1)[0]
        action = random.choice(actions)

        next_state = action
        max_q = max(Q[next_state])

        # Bellman update
        Q[state][action] = R[state][action] + gamma * max_q

        state = next_state
        if state == 2 or state == 3:
            break  # terminal states

print("Trained Q-matrix:")
print(Q / np.max(Q) * 100)  # normalize


In [ ]:
V(s) = maxₐ [R(s, a) + γ * V(s′)]:
Bellman Optimality Equation – best expected value of all possible actions.

Q(s, a): Quality of taking action a in state s.
R: Immediate reward.
γ: Discount factor.
α: Learning rate.

Q(s, a) ← Q(s, a) + α [R + γ * max Q(s′, a′) – Q(s, a)]